In [53]:
# import dependencies
import gspread
import json
import geojson
import pandas as pd
import uuid
# import requests
from pathlib import Path
from geographiclib.geodesic import Geodesic
import openrouteservice
from geopy.geocoders import Nominatim
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from time import sleep

- [Overview](#overview)
- [Activity](#activity)
- [Routes](#routes)

In [54]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("api_keys.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key("12L4EkdRqaQ_e42fGHWaTmgCeqQrNgjTfoeAEc5AB6tw")

## Overview

[Back to Top](#)

In [55]:
# get data from google sheet
overview_sheet = spreadsheet.worksheet("Overview")
overview_data = overview_sheet.get_all_records()

In [56]:
# convert lists stored as strings into actual lists
def parse_list(value):
    if isinstance(value, str) and value.strip():
        return [item.strip() for item in value.split(",")]
    return []

In [57]:
# default values for missing fields
# pretty sure this is extraneous
defaults = {
    "photos": [],
    "description": "",
}

In [58]:
# process data
for entry in overview_data:
    # parse and clean up photos list
    raw_photos = entry.get("photos", "")
    entry["photos"] = [photo.strip('[]"') for photo in parse_list(raw_photos)]

    # set default values for missing keys
    # I don't think I need this
    for key, default in defaults.items():
        entry.setdefault(key, default)

In [59]:
# save as JSON for JavaScript map
with open("../docs/resources/data/overview.json", "w") as file:
    json.dump(overview_data, file, indent=2)

print("Travel data successfully saved!")

Travel data successfully saved!


## Activity

[Back to Top](#)

In [60]:
# load Activity data
activity_sheet = spreadsheet.worksheet("Activity")
activity_data = pd.DataFrame(activity_sheet.get_all_records())

# add activity IDs
activity_data["activity_id"] = [
    str(uuid.uuid4()) if pd.isna(id) or id == "" else id
    for id in activity_data.get("activity_id", [])
]

In [61]:
# geocode locations missing lat/lng
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# cache for geocoding results, to avoid repeated requests / rate limits
try:
    with open("./geocode_cache.json", "r") as cache_file:
        geocode_cache = json.load(cache_file)
except FileNotFoundError:
    geocode_cache = {}


def geocode_location(location_name):
    # check cache first
    if location_name in geocode_cache:
        return geocode_cache[location_name]["lat"], geocode_cache[location_name]["lng"]
    try:
        location = geolocator.geocode(location_name)
        if location:
            lat_lng = {"lat": location.latitude, "lng": location.longitude}
            geocode_cache[location_name] = lat_lng  # cache result
            return lat_lng["lat"], lat_lng["lng"]
        else:
            print(f"No lat/lng found for location: {location_name}")
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {location_name}: {e}")
        return pd.Series([None, None])


# geocode if lat/lng are missing
for index, row in activity_data.iterrows():
    if not row["lat"] or not row["lng"] or pd.isna(row["lat"]) or pd.isna(row["lng"]):
        lat, lng = geocode_location(row["location"])
        print(f"Geocoding {row['location']}...")
        activity_data.at[index, "lat"] = lat
        activity_data.at[index, "lng"] = lng

        # save progress every 5 requests
        if index % 5 == 0:
            activity_data.to_csv("../docs/resources/data/Location.csv", index=False)
            with open("./geocode_cache.json", "w") as cache_file:
                json.dump(geocode_cache, cache_file)

        # for Nominatim rate limits
        # sleep(1)

# save final cache
with open("./geocode_cache.json", "w") as cache_file:
    json.dump(geocode_cache, cache_file)

In [62]:
# save updated data to csv for JavaScript map
activity_data.to_csv("../docs/resources/data/activity.csv", index=False)

print("Activity data successfully saved to csv!")

Activity data successfully saved to csv!


In [63]:
# upload updated data to Google Sheets

# replace NaN or None values with empty strings for Google Sheets compatibility
activity_data = activity_data.fillna("")

# convert df to lists of lists
activity_data_list = [
    activity_data.columns.values.tolist()
] + activity_data.values.tolist()

# upload Activity sheet
activity_sheet = spreadsheet.worksheet("Activity")
try:
    activity_sheet.clear()  # clear existing data
    activity_sheet.update(values=activity_data_list, range_name="A1")  # upload new data
except Exception as e:
    print(f"Error updating Activity sheet: {e}")

print("Data successfully uploaded to Google Sheets!")

Data successfully uploaded to Google Sheets!


## Routes

[Back to top](#)

In [64]:
# load route data
route_sheet = spreadsheet.worksheet("Routes")
route_data = pd.DataFrame(route_sheet.get_all_records())

#### Add hiking routes if in Activity with geojson

In [65]:
# add Activity geojson to Routes
for index, row in activity_data.iterrows():
    # if hiking activities with a route_path not in Routes.filename
    if (
        row["activity_type"] == "hiking"  # Activity type is hiking
        and row["route_path"]  # Activity has a route_path
        and row["route_path"] not in route_data["filename"].values  # route_path not in Routes.filename
    ):
        # add new route to Routes
        new_route = {
            "start_location": row["name"],  # Activity.name
            "end_location": row["name"],  # Activity.name
            "transport_mode": "hike",  # Fixed value
            "filename": row["route_path"],  # Activity.route_path
        }
        route_data = pd.concat([route_data, pd.DataFrame([new_route])], ignore_index=True)


In [66]:
# add route IDs
route_data["route_id"] = [
    str(uuid.uuid4()) if pd.isna(id) or id == "" else id
    for id in route_data.get("route_id", [])
]

#### Generates Airplane great circles

In [ ]:
# airplane routes

# calculate great circle route
def calculate_great_circle(start_coords, end_coords, num_points=100):
    geod = Geodesic.WGS84
    line = geod.InverseLine(start_coords[0], start_coords[1], end_coords[0], end_coords[1])
    
    # intermediate points along the great circle
    points = []
    for i in range(num_points + 1):
        s = i * line.s13 / num_points
        position = line.Position(s)
        points.append((position["lon2"], position["lat2"]))  # (longitude, latitude)
    
    return points

# save as GeoJSON
def save_great_circle_as_geojson(route_coords, output_file):
    feature = geojson.Feature(
        geometry=geojson.LineString(route_coords),
        properties={"transport_mode": "plane"}
    )
    feature_collection = geojson.FeatureCollection([feature])
    with open(output_file, "w") as f:
        geojson.dump(feature_collection, f)
    print(f"Great circle route saved to {output_file}")

# process routes from Google Sheet
def process_routes_from_sheet(route_data):
    for index, row in route_data.iterrows():
        transport_mode = row["transport_mode"].lower()
        start_location = row["start_location"]
        end_location = row["end_location"]
        filename = row.get("filename", "").strip() 

        # check if "plane" and filename is empty
        if transport_mode == "plane" and not filename:
            print(f"Processing plane route: {start_location} -> {end_location}")

            # geocode start and end locations
            geolocator = Nominatim(user_agent="geoapi", timeout=10)
            start_coords = geolocator.geocode(start_location)
            end_coords = geolocator.geocode(end_location)

            if start_coords and end_coords:
                # get latitude and longitude
                start_coords = (start_coords.latitude, start_coords.longitude)
                end_coords = (end_coords.latitude, end_coords.longitude)

                # get great circle route
                route_coords = calculate_great_circle(start_coords, end_coords)

                # save as GeoJSON
                filename = f"great_circle_route_{index}.geojson"
                save_great_circle_as_geojson(route_coords, f"../docs/resources/geojson/{filename}")

                # Update the Google Sheet with the filename, adjusting for 1-based index and header row
                route_sheet.update_cell(index + 2, route_data.columns.get_loc("filename") + 1, filename)
                print(f"Updated Google Sheet with filename: {filename}")
            else:
                print(f"Failed to geocode one or both locations: {start_location}, {end_location}")
        # else:
            # print(f"Skipping non-plane route: {start_location} -> {end_location}")

# call function to process routes
process_routes_from_sheet(route_data)

Skipping non-plane route: Jaipur -> New Delhi
Skipping non-plane route: New Delhi -> Amritsar
Skipping non-plane route: New Delhi -> Agra
Skipping non-plane route: Jaipur -> Agra
Skipping non-plane route: Calicut -> Fort Cochin
Skipping non-plane route: Fort Cochin -> Alleppey
Skipping non-plane route: Allepy -> Bangalore
Skipping non-plane route: Bangalore -> Mysore
Skipping non-plane route: Mysore -> Gokarna
Skipping non-plane route: Gokarna -> Panaji
Skipping non-plane route: Bangalore -> Anjuna
Skipping non-plane route: Anjuna -> Panaji
Skipping non-plane route: Jaipur -> Pushkar
Skipping non-plane route: Jaipur -> Jodhpur
Skipping non-plane route: Jaipur -> Alwar
Skipping non-plane route: Jaipur -> Bikaner
Skipping non-plane route: Jaipur -> Udaipur
Skipping non-plane route: Udaipur -> Jodhpur
Skipping non-plane route: Amritsar -> Manali, HImachal Pradesh
Skipping non-plane route: Manali, HImachal Pradesh -> New Delhi
Skipping non-plane route: Table Mountain -> Table Mountain
Skip

#### Generates Automobile routes, will fake a Train route with an auto route

In [68]:
# connect to OpenRouteService API for automobile routes
with open("api_keys.json") as f:
    api_keys = json.load(f)
ors_client = openrouteservice.Client(key=api_keys["openrouteservice"])

In [69]:
# route function
def fetch_route(start_coords, end_coords, transport_mode):
    try:
        profile = {
            "auto": "driving-car",
            # ORS doesn't support trains - find alterative API or method
            "train": "driving-car",
        }.get(transport_mode, "driving-car")

        # request route from ors api
        route = ors_client.directions(
            coordinates=[start_coords, end_coords], profile=profile, format="geojson"
        )
        return route
    except Exception as e:
        print(f"Failed to fetch route for mode {transport_mode}: {e}")
        return None

In [70]:
# geocode coordinates a location
def geocode_location(location):
    try:
        geocode_response = ors_client.pelias_search(text=location)
        if geocode_response["features"]:
            coords = geocode_response["features"][0]["geometry"]["coordinates"]
            return coords[0], coords[1]  # Return (lon, lat)
    except Exception as e:
        print(f"Failed to geocode location {location}: {e}")
    return None, None

In [71]:
# process GeoJSON routes and save to file
for index, row in route_data.iterrows():
    # skip extant filenames
    if pd.notna(row["filename"]) and row["filename"].strip() != "":
        # print(f"Skipping route {row['route_id']} as it already has a filename: {row['filename']}")
        continue

    # geocode start and end locations
    start_coords = geocode_location(row["start_location"])
    end_coords = geocode_location(row["end_location"])

    # if geocoding was successful, fetch the route
    if None not in start_coords and None not in end_coords:
        route = fetch_route(start_coords, end_coords, row["transport_mode"])

        # if route, save to GeoJSON file, update route_data df
        if route:
            geojson_filename = f"{row['route_id']}.geojson"
            geojson_path = Path("../docs/resources/geojson/") / geojson_filename

            with open(geojson_path, "w") as f:
                json.dump(route, f, indent=4)

            route_data.at[index, "filename"] = geojson_filename
            print(
                f"Generated GeoJSON for route {row['start_location']} to {row['end_location']}, and saved to {geojson_path}"
            )

#### Save to csv, updates google sheets

In [72]:
# save route data to csv
route_data.to_csv("../docs/resources/data/routes.csv", index=False)
print("Route data successfully saved to csv!")

Route data successfully saved to csv!


In [73]:
# upload df back to google sheets
set_with_dataframe(route_sheet, route_data)
print("Route data successfully uploaded to Google Sheets!")

Route data successfully uploaded to Google Sheets!
